<div class="alert alert-info alertinfo" style="margin-top: 0px">
<h1>  PART1. SELECT - SQL Tools (SQL Server Management studio) </h1>
</div>

<div class="alert-danger" style="margin-top: 0px">
<h1> Read me first </h1>
</div>
This are my notes from a book: "Ben, Forta. SQL in 10 Minutes, Sams Teach Yourself. Pearson Education. Kindle Edition" for quick "copy and paste" or simply to check the syntax of sql statements. 
The DBMS that I'm using is SQL Server Management studio so if you work in different DBMS some syntax might be different.

<div class="alert-success" style="margin-top: 0px">
<h1> Retrieving Data </h1>
</div>

In [ ]:
SELECT prod_id, prod_name, prod_price
FROM Products;

In [ ]:
# Caution: Can’t Be Partially DISTINCT -it applies to all columns, not just the one it precedes. 
SELECT DISTINCT vend_id, prod_name
FROM Products;

In [ ]:
SELECT TOP 5 prod_name
FROM Products; 

In [ ]:
# DOESN'T WORK IN SSMS!!!
SELECT prod_name
FROM Products
LIMIT 5 OFFSET 5;

<div class="alert-success" style="margin-top: 0px">
<h1> ORDER BY </h1>
</div>

In [ ]:
# The following code retrieves three columns and sorts the results by two of them—
# first by price (in descending order) and then by name (in ascending order as ASC is default).
SELECT prod_id, prod_price, prod_name
FROM Products
ORDER BY prod_price DESC, prod_name;

<div class="alert-success" style="margin-top: 0px">
<h1> WHERE </h1>
</div>

In [ ]:
# BETWEEN
SELECT prod_name, prod_price
FROM Products
WHERE prod_price BETWEEN 5 AND 10;

In [ ]:
# IS NULL
SELECT cust_name
FROM Customers
WHERE cust_email IS NULL;

In [ ]:
# IN
SELECT prod_name, prod_price
FROM Products
WHERE vend_id  IN ('DLL01','BRS01')
ORDER BY prod_name;

In [ ]:
# NOT
SELECT prod_name
FROM Products
WHERE NOT vend_id  = 'DLL01'
ORDER BY prod_name;

In [ ]:
# LIKE (in SMSS is not case sensitive)
SELECT Name, Town
FROM [dbo].[BusinessInformation]
WHERE Town LIKE 'bed%'

# LIKE (in SMSS [JM]% stands for 'first letter is not J nor M')
SELECT cust_contact
FROM Customers
WHERE cust_contact LIKE '[^JM]%'
ORDER BY cust_contact;

<div class="alert-success" style="margin-top: 0px">
<h1> CALCULATED FIELDS </h1>
</div>

In [ ]:
SELECT vend_name + ' (' + vend_country + ')' AS vend_title
FROM Vendors
ORDER BY vend_name;

In [ ]:
SELECT RTRIM(vend_name) + ' (' + RTRIM(vend_country) + ')' AS vend_title
FROM Vendors
ORDER BY vend_name;

In [ ]:
SELECT prod_id,
       quantity,
       item_price,
       quantity*item_price AS expanded_price
FROM OrderItems
WHERE order_num = 20008;

<div class="alert-success" style="margin-top: 0px">
<h1> AGGREGATE FUNCTIONS </h1>
</div>

In [ ]:
# aggregate functions
AVG()
COUNT()
MAX()
MIN()
SUM()

In [ ]:
SELECT AVG(prod_price) AS avg_price
FROM Products
WHERE vend_id = 'DLL01';

In [ ]:
SELECT COUNT(*) AS num_cust # will count nans (DISTINCT will not work)
FROM Customers;

SELECT COUNT(cust_email) AS num_cust # will not count nans (DISTINCT will work)
FROM Customers;

<div class="alert-success" style="margin-top: 0px">
<h1> OTHER FUNCTIONS </h1>
</div>

In [ ]:
# string functions
SUBSTRING()
LEFT()
LENGTH() DATALENGTH() LEN()
LOWER()
RTRIM() LTRIM() TRIM()
SOUNDEX()
UPPER()

In [ ]:
# numeric functions
ABS()
COS()
EXP()
PI()
SIN()
SQRT()
TAN()

In [ ]:
# date functions
GETDATE()
DATEPART()

In [ ]:
# data type conversion
CONVERT()

In [ ]:
# Example: retrieve a list of all orders made in 2012 
SELECT order_num
FROM Orders
WHERE DATEPART(yy, order_date) = 2012;

<div class="alert-success" style="margin-top: 0px">
<h1> GROUP BY </h1>
</div>

- GROUP BY clauses can contain as many columns as you want 
- you won’t get data back for each individual column level. 
- Every column listed in GROUP BY must be a retrieved column or a valid expression (but not an aggregate function)
- Aliases cannot be used.
- Most SQL implementations do not allow GROUP BY columns with variable length datatypes (such as text or memo fields).
- Aside from the aggregate calculations statements, every column in your SELECT statement must be present in the GROUP BY clause. 
- If the grouping column contains a row with a NULL value, all nulls will be together as one group. 
- The GROUP BY clause must come after any WHERE clause and before any ORDER BY clause.
- SMSS supports an optional ALL clause within GROUP BY. This clause can be used to return all groups, even those that have no matching rows (in which case the aggregate would return NULL).
- to filter groups we use HAVING instead of WHERE.
- As a rule, anytime you use a GROUP BY clause, you should also specify an ORDER BY clause. That is the only way to ensure that data will be sorted properly. Never rely on GROUP BY to sort your data.

WHERE filters before data is grouped, and HAVING filters after data is grouped. This is an important distinction; rows that are eliminated by a WHERE clause will not be included in the group. This could change the calculated values which in turn could affect which groups are filtered based on the use of those values in the HAVING clause.

In [ ]:
example which lists all vendors who have two or more products priced at 4 or more:
SELECT vend_id, COUNT(*) AS num_prods
FROM Products
WHERE prod_price >= 4
GROUP BY vend_id
HAVING COUNT(*) >= 2;

<div class="alert-success" style="margin-top: 0px">
<h1> SUBQUERRIES </h1>
</div>

- Subquery SELECT statements can only retrieve a single column. Attempting to retrieve multiple columns will return an error.
- using subqueries is not always the most efficient way to perform this type of data retrieval. Joins are more efficient

In [ ]:
example structure:
    order_items has columns ['order_number','product_id']
    orders has columns ['order_number','customer_id','order_date']
    customers has columns ['customer_id','customer_name','customer_contact']

In [ ]:
example 1: WE WANT A LIST OF ALL CUSTOMERS WHO ORDERED RGAN01

    1. Retrieve the order numbers of all orders containing item RGAN01.

        SELECT order_num
        FROM OrderItems
        WHERE prod_id = 'RGAN01';

    2. Retrieve the customer ID of all the customers who have orders listed in the order numbers returned in the previous step.

        SELECT cust_id
        FROM Orders
        WHERE order_num IN (SELECT order_num
                            FROM OrderItems
                            WHERE prod_id = 'RGAN01');

    3. Retrieve the customer information for all the customer IDs returned in the previous step.    

        SELECT customer_name, customer_contact
        FROM customers
        WHERE customer_id IN (SELECT cust_id
                                FROM Orders
                                WHERE order_num IN(SELECT order_num
                                                    FROM OrderItems
                                                    WHERE prod_id = 'RGAN01'));

In [ ]:
# The same example solved more efficiently by JOIN
SELECT c.cust_name, c.cust_contact
FROM Customers c, Orders o, OrderItems oi
WHERE c.cust_id = o.cust_id
     AND oi.order_num = o.order_num
     AND prod_id = 'RGAN01';

In [ ]:
example structure:
    order_items has columns ['order_number','product_id']
    orders has columns ['order_number','customer_id','order_date']
    customers has columns ['customer_id','customer_name','customer_contact']

In [ ]:
Example 2: WE WANT THE TOTAL NUMBER OF ORDERS PLACED BY EVERY CUSTOMER IN CUSTOMERS TABLE

    1. For each customer we get count by:
        
        SELECT COUNT(*) AS orders
        FROM Orders
        WHERE cust_id = '1000000001';

    2. For every customer we get count by:
    
        SELECT 
            cust_name
            , customer_contact
            , (SELECT COUNT(*)
                FROM Orders
                WHERE Orders.cust_id = Customers.cust_id) 
                AS orders
        FROM Customers
        ORDER BY cust_name;

In [ ]:
# The same example solved more efficiently by JOIN
SELECT
    c.cust_name
    ,c.customer_contact
    ,count(orders.*) '#Orders' # (orders.*) means count(*) from orders table
FROM Customers c JOIN Orders o
ON o.cust_id = c.cust_id
Group by c.cust_name, c.customer_contact

<div class="alert-success" style="margin-top: 0px">
<h1> JOINS </h1>
</div>

# CROSS JOIN (Cartesian Product)

In [ ]:
'''
Without the WHERE clause,
every row in the first table will be paired with every row in the second table,
regardless of if they logically go together or not.
'''

SELECT vend_name, prod_name, prod_price
FROM Vendors, Products

# INNER JOIN 

In [ ]:
SELECT v.vend_name, p.prod_name, p.prod_price
FROM Vendors v, Products p
WHERE v.vend_id = p.vend_id;

or

SELECT vend_name, prod_name, prod_price
FROM Vendors INNER JOIN Products
 ON Vendors.vend_id = Products.vend_id;

In [ ]:
# multiple tables
SELECT p.prod_name, v.vend_name, p.prod_price, o.quantity
FROM OrderItems o, Products p, Vendors v
WHERE p.vend_id = v.vend_id
     AND o.prod_id = p.prod_id
     AND order_num = 20007;

# SELF JOIN

In [ ]:
'''
example:
get all the customer contacts 
who work for the same company for which Jim Jones works.
'''

# using subquerry
SELECT cust_id, cust_name, cust_contact
FROM Customers
WHERE cust_name = (SELECT cust_name
                   FROM Customers
                   WHERE cust_contact = 'Jim Jones');

# using selfjoin
SELECT c1.cust_id, c1.cust_name, c1.cust_contact
FROM Customers AS c1, Customers AS c2
WHERE c1.cust_name = c2.cust_name
     AND c2.cust_contact = 'Jim Jones';

# JOINS + AGGREGATE

In [ ]:
SELECT Customers.cust_id,
       COUNT(Orders.order_num) AS num_ord
FROM Customers INNER JOIN Orders
     ON Customers.cust_id = Orders.cust_id
GROUP BY Customers.cust_id;

<div class="alert-success" style="margin-top: 0px">
<h1> UNION </h1>
</div>

- To return similarly structured data from different tables in a single query
- To perform multiple queries against a single table returning the data as one query
- Each query in a UNION must contain the same columns, expressions, or aggregate functions (and some DBMSs even require that columns be listed in the same order)
- Column datatypes must be compatible: They need not be the exact same type, but they must be of a type that the DBMS can implicitly convert (for example, different numeric types or different date types).
- UNION ALL to include duplicates

Ben, Forta. SQL in 10 Minutes, Sams Teach Yourself (p. 141). Pearson Education. Kindle Edition. 

Ben, Forta. SQL in 10 Minutes, Sams Teach Yourself (p. 141). Pearson Education. Kindle Edition.

In [ ]:
SELECT cust_name, cust_contact, cust_email
FROM Customers
WHERE cust_state IN ('IL','IN','MI')
    UNION
SELECT cust_name, cust_contact, cust_email
FROM Customers
WHERE cust_name = 'Fun4All'; 